In [1]:
import os, sys
path = os.getcwd()+"/../.."
print(path)
sys.path.append(path)    #give here the path to the folder containing the pac
# kage,default does not work!!

import numpy as np
import pandas as pd

from ensembler.system import basic_system as system, perturbed_system as perturbed
from ensembler.integrator import  stochastic as integ
import ensembler.potentials.OneD as pot

test_performance_sim_steps = 100

C:\Users\benja\OneDrive - ETH Zürich\PhD\Code\ensembler_public\examples\dev/../..


# Systems

Systems are containers, that contain everything, that is needed to carry out a particle simulation 

## Basic System

In [2]:
#Build a system:
## Potential
V = pot.harmonicOscillator(k=5.0)
## Integrator
integrator = integ.metropolisMonteCarloIntegrator()
## System    
sys = system.system(potential=V, integrator=integrator)

AttributeError: module 'ensembler.potentials.OneD' has no attribute 'harmonicOscillator'

In [ ]:
%time sys.simulate(test_performance_sim_steps)

In [ ]:
sys.trajectory

## Perturbed Systems

In [ ]:

integrator = integ.metropolisMonteCarloIntegrator()

sys = perturbed.perturbedSystem(lam=0.0, potential=pot.linearCoupledPotentials(), integrator=integrator)

In [ ]:
%time sys.simulate(test_performance_sim_steps)

In [ ]:
sys.trajectory

### Test different lambdas

In [ ]:
integrator = integ.metropolisMonteCarloIntegrator()
sys=[system.perturbedSystem(lam=0.0, potential=pot.linearCoupledPotentials(), integrator=integrator), 
     system.perturbedSystem(lam=1.0, potential=pot.linearCoupledPotentials(), integrator=integrator)]


In [ ]:
traj=[]
for s in sys:
    s.set_position(0.0)
    s.simulate(1000, withdrawTraj=True)
    
    t= s.trajectory
    t.position = np.array(t.position.values.tolist()).flatten()
    t.dhdlam =  pd.Series(s.potential._calculate_dVdlam(np.array(t.position.values.tolist()).flatten()))
    traj.append(t)


In [ ]:
import matplotlib.pyplot  as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colorbar
%matplotlib inline

fig, ax=plt.subplots(3, 2, figsize=[15,10])
labels=['Position /nm', 'Potential Energy /$\mathrm{kJ}\, \mathrm{mol}^{-1}$', '$ \partial \mathcal{H} \
/ \partial \lambda  \, / \mathrm{kJ} \, \mathrm{mol}^{-1}$']
for s in range(2):
    hist=[]
    for i, j in enumerate(['position', 'totPotEnergy', ]): #'dhdlam']):
        hist.append(np.histogram(traj[s][j][0], bins=30, density=1))
        ax[i,0].plot(traj[s].index, traj[s][j],'.', label='$\lambda={:d}$'.format(s))
        ax[i,0].set_ylabel(labels[i])
        ax[i,1].plot(hist[i][0], hist[i][1][:-1]+(hist[i][1][1]-hist[i][1][0])/2.0,label='$\lambda={:d}$'.format(s))
        ax[i,1].legend()
ax[2,0].set_xlabel('Step')
ax[2,1].set_xlabel('Density')
ax[2,1].set_ylim(0,20)
ax[2,1].legend()

## Enveloped System

In [ ]:
#Build a system:
## Potential
w1 = pot.harmonicOscillator(x_shift=-2)
w2 = pot.harmonicOscillator(x_shift=2)
V = pot.envelopedPotential(V_is=[w1,w2], Eoff_i=[0,0])

## Integrator
integrator = integ.metropolisMonteCarloIntegrator()
## System    
sys = system.edsSystem(potential=V, integrator=integrator)

In [ ]:
%time sys.simulate(test_performance_sim_steps)

In [ ]:
sys.trajectory